In [ ]:
patience_url = "https://rpo.library.utoronto.ca/content/patience"
sir_gawain_url = "https://rpo.library.utoronto.ca/content/sir-gawain-and-green-knight"
cleanness_url = "https://rpo.library.utoronto.ca/content/cleanness"
pearl_url = "pearl.html"
# pearl_url = "https://quod.lib.umich.edu/c/cme/Pearl?rgn=main;view=fulltext"
# need to manually download pearl html file for this step 

In [ ]:
import requests
from bs4 import BeautifulSoup

def url_to_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup

# Pearl

In [ ]:
with open("pearl.html", "r", encoding="utf-8") as pearl_file:
    pearl_html = pearl_file.read()
    pearl_soup = BeautifulSoup(pearl_html, "html.parser")

In [ ]:
pearl_stanzas = [
    "".join([
        line.text for line in stanza.find_all("span", "line")]
           ).replace("\n\n", "\n") for stanza in pearl_soup.find_all('div', "lg lg--stanza")]

# Gawain

In [ ]:
sir_gawain_soup = url_to_soup(sir_gawain_url)

In [ ]:
def soup_to_stanzas(soup):    
    stanzas = []
    current_stanza = []
    for div in soup.find_all('div', 'poemline'):
        if 'stanza' in div.get('class', []):  # Start of a new stanza section
            if current_stanza:  # If we have lines from previous stanza, add them
                stanzas.append('\n'.join(current_stanza))
            current_stanza = []
        line = div.find('span', 'line-text')
        if line and not line.find('nonum') and line.text.strip():
            current_stanza.append(line.text.strip())
    # Don't forget to add the last stanza
    if current_stanza:
        stanzas.append('\n'.join(current_stanza))
    return stanzas

In [ ]:
def url_to_stanzas(url):
    soup = url_to_soup(url)
    stanzas = soup_to_stanzas(soup)
    return stanzas

In [ ]:
gawain_stanzas = soup_to_stanzas(sir_gawain_soup)

# Cleanness

In [ ]:
import numpy as np

In [ ]:
cleanness_stanzas = url_to_stanzas(cleanness_url)

In [ ]:
# cleanness_stanzas[0][:1000]

In [ ]:
def poem_chunker(poem, num_chunks):
    # Turn 
    poem_lines = sum([poem[index_stanza].split("\n") for index_stanza in range(len(poem))], [])
    poem_chunks = ["\n".join(list([str(line) for line in chunk])) for chunk in np.array_split(poem_lines, num_chunks)]
    return poem_chunks

In [ ]:
cleanness_stanzas = poem_chunker(cleanness_stanzas, 180)

In [ ]:
# cleanness_stanzas

# Patience

In [ ]:
patience_stanzas = url_to_stanzas(patience_url)

In [ ]:
patience_stanzas = poem_chunker(patience_stanzas, 53)

In [ ]:
# patience_stanzas

# How many stanzas?

In [ ]:
(267 * len(gawain_stanzas)) + (152 * len(pearl_stanzas)) + (188 * len(cleanness_stanzas)) + (203 * len(patience_stanzas))

In [ ]:
sum([len(stanzas) for stanzas in [pearl_stanzas, gawain_stanzas, cleanness_stanzas, patience_stanzas]])

In [ ]:
# sum([sum(sum([len(stanza.split()) for stanza in stanzas], [])) /or s/tanzas in [pearl_stanzas, gawain_stanzas, cleanness_stanzas, patience_stanzas]])

## JSON conversion

In [ ]:
import json

In [ ]:
def stanzas_to_jsons(stanzas):
    stanza_dictionary = [{stanza: stanza} for stanza in stanzas]
    return stanza_dictionary

---

# GRAVEYARD

```

# gawain_stanzas[0]

np.array_split(cleanness_stanzas[0].split("\n"), 180)

# patience_soup, sir_gawain_soup, cleanness_soup, pearl_soup = map(url_to_soup,
#     [patience_url, sir_gawain_url, cleanness_url, pearl_url])

# pearl_soup

# # print(stanzas_to_jsons(cleanness_stanzas))







# len(patience_stanzas)

# len(patience_stanzas[0])

# len(gawain_stanzas)

# len(gawain_stanzas2)



## <div class="lg lg--stanza">
## code doesn't work if this is included... 

print("".join([line.text for line in stanzas[0].find_all("span", "line")]).replace("\n\n", "\n"))

pearl_stanzas

print(pearl_stanzas[-5])

# sir_gawain_soup

# pearl_stanzas

divs = sir_gawain_soup.find_all('div')
print("All unique div classes:")
classes_seen = set()
for div in divs:
    if div.get('class'):
        class_str = ' '.join(div.get('class'))
        if class_str not in classes_seen:
            classes_seen.add(class_str)
            print(f"Class: {class_str}")
print("\nTotal unique class combinations:", len(classes_seen))

print("First 10 poem lines:")
for i, line in enumerate(poem_lines[:10]):
    text = line.text.strip()
    print(f"Line {i+1}: {text}")

stanza_check = sir_gawain_soup.find_all('div', class_='poemline stanza')
print(f"\nFound {len(stanzas)} stanzas")
if stanzas:
    print("\nFirst stanza:")
    print(stanza_check[0].text.strip())

gawain_stanzas = sir_gawain_soup.find_all('div', "poemline stanza")

gawain_stanzas2 = [
    "\n".join([
        line.text for line in stanza.find_all("span", "line-text")
        if not line.find('nonum') and line.text.strip()  # Skip translations and empty lines
    ]).replace("\n\n", "\n")
    for stanza in sir_gawain_soup.find_all('div', "poemline stanza")
    if any(line.text.strip() for line in stanza.find_all("span", "line-text") if not line.find('nonum'))  # Only include stanzas with non-empty content
]

gawain_stanzas2

print(gawain_stanzas2[0])

print(gawain_stanzas2[0])

print(gawain_stanzas2[1])





print("".join([line.text for line in gawain_stanzas[1].find_all("span", "line-text")]).replace("\n\n", "\n"))